# Generate All Paper Figures for "The Inaccessible Game"

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/lawrennd/tig-code/blob/main/examples/generate_all_paper_figures.ipynb)

## Neil D. Lawrence

### November 2025

This notebook generates all figures referenced in the paper "The Inaccessible Game". 

**Paper figures generated:**
1. `fig-n3_coldness-vs-component-norms.pdf` - Component norms vs inverse temperature (N=3 binary)
2. `fig-n3_coldness-vs-ratio.pdf` - Ratio ||A||/||S|| vs inverse temperature (N=3 binary)
3. `fig-n3_trajectory-comparison.pdf` - Parameter trajectories: constrained vs unconstrained
4. `fig-n3_trajectory-interaction-comparison.pdf` - Interaction parameter trajectories
5. `fig-critical-scaling_magnetisation-vs-temperature.pdf` - Magnetisation vs temperature (Curie-Weiss)
6. `fig-curie-weiss_multi-inform-gradient-vs-n.pdf` - Multi-information gradient scaling (Curie-Weiss)

## Setup and Configuration

In [ ]:
# Auto-install TIG package if not available
import os

try:
    import tig
except ImportError:
    print("📦 Installing TIG package...")
    %pip install -q git+https://github.com/lawrennd/tig-code.git
    print("✓ TIG package installed!")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tig import generic_decomposition_n3 as gd
from tig import curie_weiss_equivalence as cw

In [ ]:
# Set plot style consistently
plt.style.use('seaborn-v0_8-whitegrid' if 'seaborn-v0_8-whitegrid' in plt.style.available else 'default')
big_wide_figsize = (10, 5)
big_figsize = (8, 8)
plt.rcParams.update({
    'font.size': 14,
    'font.family': 'serif',
    'axes.labelsize': 18,
    'xtick.labelsize': 16,
    'ytick.labelsize': 16,
    'legend.fontsize': 'medium',
})

# Create diagrams directory if it doesn't exist
os.makedirs('./diagrams', exist_ok=True)

print("✓ Plot configuration complete")
print("✓ Diagrams directory ready")

---
# Part 1: N=3 Binary Variable Experiments

These experiments validate the GENERIC-like decomposition for a simple three-variable binary system (Section 4.3 in paper).

## Figure 1a & 1b: Temperature Dependence of GENERIC Components

In [ ]:
# Initialize frustrated system
N = 3
h = 0  # No external field
theta_frustrated = np.array([h, h, h, 1, -1, 1])  # Frustrated parameters
theta_frustrated = theta_frustrated / np.sqrt(theta_frustrated.T @ theta_frustrated)  # Normalize

print(f"System: {N} binary variables with frustrated interactions")
print(f"Parameters: θ = {theta_frustrated}")

In [ ]:
# Vary temperature and compute GENERIC decomposition
theta = theta_frustrated
temp_values = np.logspace(-1.5, 1.5, 200)
beta_values = 1 / temp_values
ratios_temp = []
norms_S_temp = []
norms_A_temp = []

print("Computing GENERIC decomposition across temperatures...")
for i, beta in enumerate(beta_values):
    theta_scaled = beta * theta
    try:
        result_temp = gd.analyse_generic_structure(theta_scaled, N)
        ratios_temp.append(result_temp['ratio'])
        norms_S_temp.append(result_temp['norm_S'])
        norms_A_temp.append(result_temp['norm_A'])
        if (i+1) % 50 == 0:
            print(f"  Progress: {i+1}/{len(beta_values)}")
    except:
        ratios_temp.append(np.nan)
        norms_S_temp.append(np.nan)
        norms_A_temp.append(np.nan)

ratios_temp = np.array(ratios_temp)
norms_S_temp = np.array(norms_S_temp)
norms_A_temp = np.array(norms_A_temp)

# Find peak
valid_mask = ~np.isnan(ratios_temp)
peak_idx = np.argmax(ratios_temp[valid_mask])
peak_beta = beta_values[valid_mask][peak_idx]
peak_ratio = ratios_temp[valid_mask][peak_idx]

print(f"\n✓ Computation complete")
print(f"  Peak ratio: ||A||/||S|| = {peak_ratio:.3f} at β = {peak_beta:.2f}")

In [ ]:
# FIGURE 1a: Component norms vs temperature
fig, ax = plt.subplots(figsize=big_wide_figsize)
ax.semilogx(temp_values[valid_mask], norms_S_temp[valid_mask], 'r--', linewidth=4,
            label=r'$\|S\|$ (dissipative)')
ax.semilogx(temp_values[valid_mask], norms_A_temp[valid_mask], 'b-', linewidth=4,
            label=r'$\|A\|$ (conservative)')
ax.set_xlabel(r'inverse temperature $\beta$')
ax.set_ylabel('Frobenius Norm')
ax.legend(fontsize=12)
ax.grid(True, alpha=0.3)

plt.savefig('./diagrams/fig-n3_coldness-vs-component-norms.pdf', bbox_inches='tight', dpi=300)
print("✓ Saved: fig-n3_coldness-vs-component-norms.pdf")
plt.show()

In [ ]:
# FIGURE 1b: Ratio vs inverse temperature
fig, ax = plt.subplots(figsize=big_wide_figsize)
ax.semilogx(temp_values[valid_mask], ratios_temp[valid_mask], 'b-', linewidth=4)
peak_temp = 1.0 / peak_beta
ax.axvline(peak_temp, color='r', linestyle='--', linewidth=2, alpha=0.5)
ax.set_xlabel(r'inverse temperature $\beta$')
ax.set_ylabel(r'$\|A\|/\|S\|$')
ax.grid(True, alpha=0.3)
ax.text(peak_temp, peak_ratio*1.1, f'Peak: β={peak_temp:.2f}',
        ha='center', fontsize=16)

plt.savefig('./diagrams/fig-n3_coldness-vs-ratio.pdf', bbox_inches='tight', dpi=300)
print("✓ Saved: fig-n3_coldness-vs-ratio.pdf")
plt.show()

## Figures 2a & 2b: Constrained vs Unconstrained Dynamics

In [ ]:
# Solve constrained dynamics
print("Solving constrained dynamics...")
sol_constrained = gd.solve_constrained_maxent(
    theta_frustrated, N=3, n_steps=40000, dt=0.01, verbose=False
)
print(f"  Converged: {sol_constrained['converged']}")
print(f"  Final flow norm: {sol_constrained['flow_norms'][-1]:.2e}")

# Solve unconstrained dynamics
print("\nSolving unconstrained dynamics...")
sol_unconstrained = gd.solve_unconstrained_maxent(
    theta_frustrated, N=3, n_steps=40000, dt=0.01, verbose=False
)
print(f"  Converged: {sol_unconstrained['converged']}")
print(f"  Final flow norm: {sol_unconstrained['flow_norms'][-1]:.2e}")
print("\n✓ Both trajectories computed")

In [ ]:
# FIGURE 2a: Parameter space trajectories (θ₁, θ₂)
traj_c = sol_constrained['trajectory']
traj_u = sol_unconstrained['trajectory']

fig, ax = plt.subplots(figsize=big_figsize)

# Plot trajectories
ax.plot(traj_u[:, 0], traj_u[:, 1], 'r--', alpha=0.6, linewidth=4,
        label='Unconstrained')
ax.plot(traj_c[:, 0], traj_c[:, 1], 'b-', alpha=0.7, linewidth=4,
        label='Constrained')

# Mark initial and final points
ax.plot(traj_c[0, 0], traj_c[0, 1], 'go', markersize=12,
        label='Initial', zorder=5)
ax.plot(traj_u[-1, 0], traj_u[-1, 1], 'r*', markersize=16, zorder=5)
ax.plot(traj_c[-1, 0], traj_c[-1, 1], 'bs', markersize=12, zorder=5)

ax.set_xlabel(r'$\theta_1$')
ax.set_ylabel(r'$\theta_2$')
ax.legend(fontsize=14)
ax.grid(True, alpha=0.3)
ax.axhline(0, color='k', linewidth=1, linestyle='--', alpha=0.3)
ax.axvline(0, color='k', linewidth=1, linestyle='--', alpha=0.3)

plt.tight_layout()
plt.savefig('./diagrams/fig-n3_trajectory-comparison.pdf', bbox_inches='tight', dpi=300)
print("✓ Saved: fig-n3_trajectory-comparison.pdf")
plt.show()

In [ ]:
# FIGURE 2b: Interaction parameter trajectories (θ₁₂, θ₁₃)
fig, ax = plt.subplots(figsize=big_figsize)

# Plot trajectories
ax.plot(traj_u[:, 3], traj_u[:, 4], 'r--', alpha=0.6, linewidth=4,
        label='Unconstrained')
ax.plot(traj_c[:, 3], traj_c[:, 4], 'b-', alpha=0.7, linewidth=4,
        label='Constrained')

# Mark initial and final points
ax.plot(traj_c[0, 3], traj_c[0, 4], 'go', markersize=12,
        label='Initial', zorder=5)
ax.plot(traj_u[-1, 3], traj_u[-1, 4], 'r*', markersize=16, zorder=5)
ax.plot(traj_c[-1, 3], traj_c[-1, 4], 'bs', markersize=12, zorder=5)

ax.set_xlabel(r'$\theta_{12}$')
ax.set_ylabel(r'$\theta_{13}$')
ax.legend(fontsize=14)
ax.grid(True, alpha=0.3)
ax.axhline(0, color='k', linewidth=1, linestyle='--', alpha=0.3)
ax.axvline(0, color='k', linewidth=1, linestyle='--', alpha=0.3)

plt.tight_layout()
plt.savefig('./diagrams/fig-n3_trajectory-interaction-comparison.pdf', bbox_inches='tight', dpi=300)
print("✓ Saved: fig-n3_trajectory-interaction-comparison.pdf")
plt.show()

---
# Part 2: Curie-Weiss Model Experiments

These experiments validate the energy-entropy equivalence theorem in the thermodynamic limit (Section 5.2 in paper).

## Figure 3: Critical Scaling of Magnetisation

Order parameter |m| transitions from 0 to finite values at βc.

In [ ]:
# Curie-Weiss parameters
J = 1.0   # Coupling strength
h_sym = 0.01  # Small external field to break symmetry
beta_c = 1.0 / J  # Critical inverse temperature
T_c = 1.0 / beta_c

n_vals = [10, 50, 100, 500, 1000]
linestyle_vals = ['solid', (0, (5, 5)), (0, (1, 1)), (0, (5, 1, 1, 1)), (0, (5, 10))]

print(f"Curie-Weiss Model: J = {J}, h = {h_sym}")
print(f"Critical temperature: βc = {beta_c}, Tc = {T_c}")

In [ ]:
# Scan temperatures around Tc
beta_range = np.linspace(0.5 * beta_c, 2.0 * beta_c, 100)
T_range = 1.0 / beta_range

results = {}
linestyle_dict = {str(n): linestyle for n, linestyle in zip(n_vals, linestyle_vals[::-1])}

print("\nComputing magnetisation vs temperature...")
for n in n_vals:
    strn = str(n)
    results[strn] = {
        'magnetisations': [],
    }
    print(f"  n = {n}...", end=" ")
    for beta in beta_range:
        m = cw.exact_expectation_magnetisation(beta, J, h_sym, n)
        results[strn]['magnetisations'].append(m)
    print("done")

print("✓ Computation complete")

In [ ]:
# FIGURE 3: Magnetisation vs temperature
fig, ax = plt.subplots(figsize=big_wide_figsize)
for n in n_vals:
    strn = str(n)
    ax.plot(beta_range / beta_c, np.abs(results[strn]['magnetisations']),
            label=rf'$n={n}$', color='g', linestyle=linestyle_dict[strn], linewidth=4)

ax.axvline(1.0, color='r', linestyle='--', label=r'$\beta_c$', linewidth=2)
ax.set_xlabel(r'$\beta / \beta_c$')
ax.set_ylabel('$|m|$')
ax.legend()
ax.grid(True, alpha=0.3)

plt.savefig('./diagrams/fig-critical-scaling_magnetisation-vs-temperature.pdf', dpi=150, bbox_inches='tight')
print("✓ Saved: fig-critical-scaling_magnetisation-vs-temperature.pdf")
plt.show()

## Figure 4: Multi-information Gradient Scaling

Shows that ∇ₘI becomes intensive (O(1)) as n → ∞, while ∇ₘH scales extensively (O(n)).

In [ ]:
# Higher resolution: logarithmically spaced n values
n_values_log = np.unique(np.logspace(np.log10(10), np.log10(10000), 40).astype(int))

# Multiple temperature points
beta_vals = [0.25*beta_c, 0.5*beta_c, 0.8*beta_c, beta_c, 1.5*beta_c, 2.0*beta_c, 4.0*beta_c]
labels = [r'$0.25\beta_c$', r'$0.5\beta_c$', r'$0.8\beta_c$',
          r'$\beta_c$ (Critical)', r'$1.5\beta_c$', r'$2.0\beta_c$', r'$4.0\beta_c$']

colors = ['#2E7D32', '#F57C00', '#D32F2F', '#1976D2', '#7B1FA2', '#00897B', '#C2185B']

results_scaling = {}
print("\nComputing multi-information gradient scaling...")
for beta, label, color in zip(beta_vals, labels, colors):
    print(f"  {label}...", end=" ")
    log_grads = []
    for n in n_values_log:
        nabla_m_I = cw.exact_gradient_multi_info_wrt_m(beta, J, h_sym, n)
        log_grads.append(np.log(np.abs(nabla_m_I)))
    results_scaling[label] = {'log_diffs': np.array(log_grads), 'color': color}
    print("done")

print("✓ Computation complete")

In [ ]:
# FIGURE 4: Multi-information gradient vs n
fig, ax = plt.subplots(figsize=big_figsize)
for label in labels:
    ax.plot(n_values_log, results_scaling[label]['log_diffs'], 'o-',
            color=results_scaling[label]['color'], label=label, markersize=4, alpha=0.8)

ax.set_xlabel('system size $n$')
ax.set_ylabel(r'$\log \left|\frac{\text{d}I}{\text{d}m}\right|$')
ax.legend(loc='best')
ax.grid(True, alpha=0.3, which='both')
ax.set_xscale("log")

plt.savefig('./diagrams/fig-curie-weiss_multi-inform-gradient-vs-n.pdf', dpi=300, bbox_inches='tight')
print("✓ Saved: fig-curie-weiss_multi-inform-gradient-vs-n.pdf")
plt.show()

---
# Summary

All paper figures have been generated and saved to the `./diagrams/` directory:

**N=3 Binary Variables (Section 4.3):**
- ✓ `fig-n3_coldness-vs-component-norms.pdf`
- ✓ `fig-n3_coldness-vs-ratio.pdf`
- ✓ `fig-n3_trajectory-comparison.pdf`
- ✓ `fig-n3_trajectory-interaction-comparison.pdf`

**Curie-Weiss Model (Section 5.2):**
- ✓ `fig-critical-scaling_magnetisation-vs-temperature.pdf`
- ✓ `fig-curie-weiss_multi-inform-gradient-vs-n.pdf`